# Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

# raw data / сырые данные

In [2]:
raw_data = pd.read_csv('movie_bd_v5.csv')
raw_data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
981,tt1735898,170000000,396600000,Snow White and the Huntsman,Kristen Stewart|Charlize Theron|Chris Hemswort...,Rupert Sanders,The Fairytale is Over,"After the Evil Queen marries the King, she per...",127,Adventure|Fantasy|Drama,Universal Pictures|Roth Films,5/30/2012,5.7,2012
1189,tt1690953,76000000,970761885,Despicable Me 2,Steve Carell|Kristen Wiig|Benjamin Bratt|Miran...,Pierre Coffin|Chris Renaud,Back 2 Work,Gru is recruited by the Anti-Villain League to...,98,Animation|Comedy|Family,Universal Pictures|Illumination Entertainment,6/25/2013,7.0,2013
1681,tt0758758,15000000,56255142,Into the Wild,Emile Hirsch|Marcia Gay Harden|William Hurt|Je...,Sean Penn,Into the heart. Into the soul.,"The true story of top student and athlete, Chr...",148,Adventure|Drama,Paramount Vantage|River Road Entertainment|Art...,9/11/2007,7.7,2007
1865,tt0161081,100000000,155464351,What Lies Beneath,Harrison Ford|Michelle Pfeiffer|Diana Scarwid|...,Robert Zemeckis,He was the perfect husband until his one mista...,When Claire Spencer starts hearing ghostly voi...,130,Drama|Horror|Mystery|Thriller,DreamWorks SKG|Twentieth Century Fox Film Corp...,7/21/2000,6.1,2000
1510,tt0401445,35000000,42064105,A Good Year,Russell Crowe|Marion Cotillard|Albert Finney|T...,Ridley Scott,Everything matures... eventually.,Failed London banker Max Skinner inherits his ...,118,Comedy|Drama|Romance,Fox 2000 Pictures|Scott Free Productions,9/9/2006,6.4,2006


## Колонки raw_data

In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


## Приводим raw_data к нормальному виду

Нормализовать след колонки, согласно 1ой нормальной формы

- `genres` : string - split('|')
- `cast`  : string - split('|')
- `director` : string - split('|')
- `production_companies` : string - split('|')
- `release_date` : string - дата MM/DD/YYYY

Добавить/вычислить колонки
- `year` : int - год релиза
- `month` : int (1..12) - месяц релиза
- `date` : int (1..31) - дата релиза
- `profit` - выручка

In [4]:
# разделяет строку по вертикальной черте, 
# возвращает массив
def split_str_as_list( g ):
    if type(g) is str:
        return g.split('|')
    return []

In [5]:
# Нормализации 1.а - одная ячейка одно значени, в частности жанры перечислены через вертикальную черту
def normalize_nf1( src_df, column, norm_f, newColumn=None, includeColumns=None ):
    """Нормализация DataFrame к 1 форме - случай split('|')
       Если в ячейки есть 2 или более значения, 
         1. то вся строка будет продублирова столько раз, сколько значейний в ячейке
         2. значение ячеек будет заменено на разультат разбияния

       Args:
         src_df - DataFrame
         column : str - Колонка содержащяя множествно значений
         norm_f( value ):List[values] - функция разбиения значения на список значений
         newColumn : str | None - переименование колонки
         includeColumns : None | List[str] - список колонок которые следует оставить

       Returs:
         Новый DataFrame

       Пример

       | a | b   |
       |===|=====|
       | 0 | 1,2 |

       После normalize_nf1( data, 'b', lambda x:x.split(',') )

       | a | b |
       |===|===|
       | 0 | 1 |
       | 0 | 2 |
    """
    if not (type(src_df) is pd.DataFrame):
        raise "!! src_df not pd.DataFrame"
    if not (type(column) is str):
        raise "!! column not str"
    if not callable(norm_f):
        raise "!! norm_f not callable"    

    # Целевая колонка не указана - используем оригинальную
    if newColumn==None:
        newColumn = column
    elif not( type(newColumn) is str):
        raise "!! newColumn must None|str"

    # Строим новый массив данных
    column_store = {}
    for col_name in src_df.columns:
        column_store[col_name] = []
        column_store[col_name if col_name!=column else newColumn] = []

    # по строчное хранение
    row_store = []        
    for index, srow in src_df.iterrows():
        raw_unsplitted_data = srow[column]
        splitted_data = norm_f(raw_unsplitted_data)
        for splitted_value in splitted_data:
            row = {}
            for col_name in src_df.columns:
                row[col_name] = srow[col_name]
                row[col_name if col_name!=column else newColumn] = srow[col_name] if col_name != column else splitted_value
            row_store.append( row )
    
    for row in row_store:
        for k in column_store.keys():
            column_store[k].append(row[k])

    df = pd.DataFrame( column_store, index=range(0,len(row_store)) )

    # Грохаем лишние колонки
    if type(includeColumns) is list:
        cols = []
        keep_cols = []
        for col_name in df.columns:
            cols.append(col_name)
            if col_name == newColumn:
                keep_cols.append(col_name)
            elif col_name in includeColumns:
                keep_cols.append(col_name)
        drop_cols = [ c for c in cols if c not in keep_cols ]
        df = df.drop( drop_cols, axis=1 )

    return df

In [6]:
normalize_nf1( raw_data.head(3), 'genres', split_str_as_list, 'genre', ['imdb_id','genres','original_title'] )

,imdb_id,original_title,genres,genre
0,tt0369610,Jurassic World,Action|Adventure|Science Fiction|Thriller,Action
1,tt0369610,Jurassic World,Action|Adventure|Science Fiction|Thriller,Adventure
2,tt0369610,Jurassic World,Action|Adventure|Science Fiction|Thriller,Science Fiction
3,tt0369610,Jurassic World,Action|Adventure|Science Fiction|Thriller,Thriller
4,tt1392190,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller,Action
5,tt1392190,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller,Adventure
6,tt1392190,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller,Science Fiction
7,tt1392190,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller,Thriller
8,tt2908446,Insurgent,Adventure|Science Fiction|Thriller,Adventure
9,tt2908446,Insurgent,Adventure|Science Fiction|Thriller,Science Fiction


## Нормализуем данные

In [7]:
genres = normalize_nf1( raw_data, 'genres', split_str_as_list, 'genre', ['imdb_id'] )
acts = normalize_nf1( raw_data, 'cast', split_str_as_list, 'actor', ['imdb_id'] )
dirs = normalize_nf1( raw_data, 'director', split_str_as_list, 'dir', ['imdb_id'] )
prods = normalize_nf1( raw_data, 'production_companies', split_str_as_list, 'prod', ['imdb_id'] )

In [8]:
# Копируем исходные данные
movies = raw_data.copy()

# Удаляем те, что уже нормализированы
movies = movies.drop( ['genres','cast','director','production_companies'], axis=1 )

# Вычисление даты из строки формата MM/DD/YYYY
def date_str_to_values( s ):
    '''
    Вычисление даты из строки формата MM/DD/YYYY
    Args:
        s : str - строка с датой в формате MM/DD/YYYY
    Returns:
        { 'y': int # год
        , 'm': int # месяц
        , 'd': int # дата
        }
    '''
    nums = list(map( lambda x:int(x), s.split('/') ))
    return { 'y':nums[2], 'm':nums[0], 'd':nums[1] }

# Вычисляем колонки year, month, date
movies['year'] = movies['release_date'].apply( lambda x: date_str_to_values(x)['y'] )
movies['month'] = movies['release_date'].apply( lambda x: date_str_to_values(x)['m'] )
movies['date'] = movies['release_date'].apply( lambda x: date_str_to_values(x)['d'] )

# Удаляем те, что уже нормализированы
movies = movies.drop( ['release_date'], axis=1 )

# Если дата(год) не совпадает с release_year - ругаемся
if( len(movies[ movies.release_year != movies.year ])>0 ):
    raise '!! movies.release_year != movies.year'

# Удаляем те, что уже нормализированы
movies = movies.drop(['release_year'], axis=1)

# Добавляем колонку profit
movies['profit'] = movies['revenue'] - movies['budget']

# Добавляем колонку original_title_len - Длина заголовка в символах
movies['original_title_len'] = movies['original_title'].apply( lambda x: len(x) )

# Подсчет кол-ва слов
# Есть не однозначность под пониманием кол-ва слов
#
# https://skillfactorydspr.slack.com/archives/C017W2EQTRV/p1610720433015800
#
# Описания фильмов какой студии в среднем самые длинные по количеству слов?
# 1. Описание - это колонка overview ?
# 2. В понятие слово входят только буквы или еще спец символы (запятые, точки - например park, это слово из 5 букв включая запятую или 4 буквы исключая запятую) ?
# 3. Слова с разделитем например Twenty-two - считам за два слова или еще пример here's ?
# 4. Слова не содержащие букв например числа - считаем за слова ?
# 5. Или мне просто разделить по пробелу ?
#
# Добрый день!
# 1. Да, это overview
# 2. Разделять только по пробелам, park, - это слово из 5 символов
# 3. Разделитель не считаем за два слово
# 4. считаем за слова
#
# Вариант 1
def words_of( text ):
    words = []
    word = ''
    for c in text:
        if c.isalpha():
            word = word + c
        else:
            if len(word)>0:
                words.append(word)
                word = ''
    return words

# Вариант 2
def simple_words_of( text ):
    return text.split(' ')

movies['overview_words_count'] = movies['overview'].apply( lambda x: len(simple_words_of(x)) )
movies

,imdb_id,budget,revenue,original_title,tagline,overview,runtime,vote_average,year,month,date,profit,original_title_len,overview_words_count
0,tt0369610,150000000,1513528810,Jurassic World,The park is open.,Twenty-two years after the events of Jurassic ...,124,6.5,2015,6,9,1363528810,14,26
1,tt1392190,150000000,378436354,Mad Max: Fury Road,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,7.1,2015,5,13,228436354,18,110
2,tt2908446,110000000,295238201,Insurgent,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,6.3,2015,3,18,185238201,9,22
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,7.5,2015,12,15,1868178225,28,26
4,tt2820852,190000000,1506249360,Furious 7,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,7.3,2015,4,1,1316249360,9,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,tt0120903,75000000,157299717,X-Men,Evolution Begins,"Two mutants, Rogue and Wolverine, come to a pr...",104,6.6,2000,7,13,82299717,5,27
1885,tt0192255,22000000,13555988,The Little Vampire,"They're not just best friends, they're blood b...","Based on the popular books, the story tells of...",95,6.4,2000,10,27,-8444012,18,75
1886,tt0131704,76000000,35134820,The Adventures of Rocky & Bullwinkle,This summer it's not the same old bull.,Rocky and Bullwinkle have been living off the ...,88,4.0,2000,6,30,-40865180,36,58
1887,tt0162983,40000000,36037909,Hanging Up,Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,5.2,2000,2,16,-3962091,10,24


In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   tagline               1889 non-null   object 
 5   overview              1889 non-null   object 
 6   runtime               1889 non-null   int64  
 7   vote_average          1889 non-null   float64
 8   year                  1889 non-null   int64  
 9   month                 1889 non-null   int64  
 10  date                  1889 non-null   int64  
 11  profit                1889 non-null   int64  
 12  original_title_len    1889 non-null   int64  
 13  overview_words_count  1889 non-null   int64  
dtypes: float64(1), int64(9), object(4)
memory usage: 206.7+ KB


# 1. У какого фильма из списка самый большой бюджет?

In [10]:
r = movies[ movies.budget == movies.budget.max() ]
r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [11]:
r = movies[ movies.runtime == movies.runtime.max() ]
r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?

In [12]:
r = movies[ movies.runtime == movies.runtime.min() ]
r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?

In [13]:
round(movies.runtime.mean())

110

# 5. Каково медианное значение длительности фильмов?

In [14]:
round(movies.runtime.median())

107

# 6. Какой самый прибыльный фильм?

In [16]:
r = movies[ movies.profit == movies.profit.max() ]
r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный?

In [17]:
r = movies[ movies.profit == movies.profit.min() ]
r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [18]:
len( movies[ movies.profit > 0 ] )

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [19]:
d = movies[ movies.year == 2008 ]
r = d[ d.revenue == d.revenue.max() ]
r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [20]:
d = movies[ (movies.year >= 2012) & (movies.year <= 2014) ]
r = d[ d.profit == d.profit.min() ]
r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [21]:
genres.describe()
genres.describe().loc['top']['genre']

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными?

In [22]:
# Обсуждение https://skillfactorydspr.slack.com/archives/C017W2EQTRV/p1610540743055200
# Обсуждение https://skillfactorydspr.slack.com/archives/C017W2EQTRV/p1610710126014500
# Постановка вопроса не правильна в корне
#   при том мысль такая же была - 
#   что популярный (top по частоте упоминания) != прибыльный (profit)
genres.genre.value_counts().index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [24]:
j = dirs.merge( movies, on='imdb_id', how='outer' )
j.groupby(['dir'])['revenue'].sum().sort_values(ascending=False).index[0]

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [25]:
j = genres.merge( movies, on='imdb_id', how='outer' )
j = j.merge( dirs, on='imdb_id', how='outer' )
j = j[ j.genre == 'Action' ]
j.groupby( ['dir'] )['genre'].count().sort_values(ascending=False).index[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году?

In [26]:
j = acts.merge( movies, on='imdb_id', how='outer' )
j = j[ j.year == 2012 ]
j.groupby(['actor'])['revenue'].sum().sort_values(ascending=False).index[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [27]:
j = acts.merge( movies, on='imdb_id', how='outer' )
# в фильмах, где бюджет выше среднего по данной выборке. 
j = j[ j.budget > j.budget.mean() ]
j.groupby('actor').count().sort_values(['imdb_id'],ascending=False).index[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage?

In [28]:
j = acts.merge( genres, on='imdb_id', how='outer' )
j = j[ j.actor=='Nicolas Cage' ].drop(['imdb_id'], axis=1)
j = j.groupby(['genre']).count().sort_values(['actor'],ascending=False)
j.index[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [29]:
j = prods.merge( movies, on='imdb_id', how='left' )
j = j.query( "prod == 'Paramount Pictures' " )
j = j[ j.profit == j.profit.min() ]
r = j
r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [30]:
d = movies.groupby(['year']).sum().sort_values(['year'],ascending=False)
d.index[0]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [31]:
# https://skillfactorydspr.slack.com/archives/C017W2EQTRV/p1607752079405900?thread_ts=1607728173.403500&cid=C017W2EQTRV
# Приветствую! Надо учесть все четыре компании с Warner Bros. в названии.
j = prods.merge( movies, on='imdb_id', how='left' )
j = j[ j['prod'].str.contains('Warner Bros') ]
j = j.groupby('year')['profit'].sum().sort_values(ascending=False)
j = j.loc[[2014,2008,2012,2010,2015]].sort_values(ascending=False)
j.index[0]

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [32]:
movies.groupby(['month']).count().sort_values(['imdb_id'],ascending=False).index[0]

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [33]:
movies.groupby(['month']).count().loc[ [8,7,6] ].imdb_id.sum()

450

# 23. Для какого режиссера зима – самое продуктивное время года?

In [34]:
j = dirs.merge( movies, on='imdb_id', how='left' )
j['winter'] = j['month'].apply( lambda x: x in [1,2,12])
j = j[ j['winter'] ].groupby('dir')['imdb_id'].count().sort_values(ascending=False)
j.index[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [35]:
# https://skillfactorydspr.slack.com/archives/C017W2EQTRV/p1610572293067300
# Под вопросом
# "Какая студия даёт самые длинные названия своим фильмам по количеству символов?"
# имеется виду
# "Какая студия даёт самые длинные названия своим фильмам по количеству символов (в среднем по всей выборке)?"
j = prods.merge( movies, on='imdb_id', how='left' )
j = j.groupby(['prod'])['original_title_len'].mean().sort_values(ascending=False)
j.index[0]

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [36]:
# overview_words_count
j = prods.merge( movies, on='imdb_id', how='left' )
j = j.groupby(['prod'])['overview_words_count'].mean().sort_values(ascending=False)
j.index[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу?

In [37]:
# 1) Inside Out, The Dark Knight, 12 Years a Slave
# 2) BloodRayne, The Adventures of Rocky & Bullwinkle
# 3) Batman Begins, The Lord of the Rings: The Return of the King, Upside Down
# 4) 300, Lucky Number Slevin, Kill Bill: Vol. 1
# 5) Upside Down, Inside Out, Iron Man 
j = movies.sort_values(['vote_average'],ascending=False)
j = j.head( int(len(j)*0.01) )

len( j[ j.original_title.str.contains('BloodRayne') ] )>0

# original_title
cases = { '1': ['Inside Out','The Dark Knight', '12 Years a Slave']
        , '2': ['BloodRayne','The Adventures of Rocky & Bullwinkle'] 
        , '3': ['Batman Begins','The Lord of the Rings: The Return of the King','Upside Down']
        , '4': ['300','Lucky Number Slevin','Kill Bill: Vol. 1']
        , '5': ['Upside Down','Inside Out','Iron Man']
        }
matched_cases = []
for k,titles in cases.items():
    fails = 0
    for title in titles:
        inTop = len(j[ j.original_title.str.contains(title) ]) > 0
        if not inTop:
            fails = fails + 1
    if fails==0:
        matched_cases.append((k,titles))
for mc in matched_cases:
    print( 'matched ',mc)

matched  ('1', ['Inside Out', 'The Dark Knight', '12 Years a Slave'])


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?

In [38]:
# Варианты
# 1) Johnny Depp & Helena Bonham Carter
# 2) Ben Stiller & Owen Wilson
# 3) Vin Diesel & Paul Walker 
# 4) Adam Sandler & Kevin James 
# 5) Daniel Radcliffe & Rupert Grint

pairs = [
    { 'count': 0, 'names': ['Johnny Depp', 'Helena Bonham Carter'] },
    { 'count': 0, 'names': ['Ben Stiller', 'Owen Wilson'] },
    { 'count': 0, 'names': ['Vin Diesel', 'Paul Walker'] },
    { 'count': 0, 'names': ['Adam Sandler', 'Kevin James'] },
    { 'count': 0, 'names': ['Daniel Radcliffe', 'Rupert Grint'] },
]

for index, srow in raw_data.iterrows():
    actors = srow.cast.split('|')
    for p in pairs:
        matched = 0
        for name in p['names']:
            if name in actors:
                matched = matched + 1
        if matched==len(p['names']):
            p['count'] = p['count'] + 1

sorted_pairs = sorted(pairs, key=lambda p:p['count'])
sorted_pairs[ len(sorted_pairs)-1 ]

{'count': 8, 'names': ['Daniel Radcliffe', 'Rupert Grint']}